<h2>importing the Necessary Packages/Libraries</h2>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<h2> Loading the CSV Files using Pandas FrameWork</h2>

In [ ]:
dim_campaigns = pd.read_csv(r"C:\Users\hruth\Desktop\AI engineer\Virtual Internship\week 1\datasets\dim_campaigns.csv")
dim_campaigns.head()

In [ ]:
dim_campaigns.dtypes

In [ ]:
dim_campaigns.info()

In [ ]:
dim_campaigns.describe()

In [ ]:
dim_campaigns.isna().sum()

In [ ]:
dim_campaigns.duplicated()

In [ ]:
dim_products = pd.read_csv(r"C:\Users\hruth\Desktop\AI engineer\Virtual Internship\week 1\datasets\dim_products.csv")
dim_products.head()

In [ ]:
dim_products.dtypes

In [ ]:
dim_products.info()

In [ ]:
dim_products.describe()

In [ ]:
dim_products.isna().sum()

In [ ]:
dim_products.duplicated().sum()

In [ ]:
dim_stores = pd.read_csv(r"C:\Users\hruth\Desktop\AI engineer\Virtual Internship\week 1\datasets\dim_stores.csv")
dim_stores.head()

In [ ]:
dim_stores.dtypes

In [ ]:
dim_stores.describe()

In [ ]:
dim_stores.info()

In [ ]:
dim_stores.duplicated().sum()

In [ ]:
fact_events = pd.read_csv(r"C:\Users\hruth\Desktop\AI engineer\Virtual Internship\week 1\datasets\fact_events.csv")
fact_events.head()

In [ ]:
fact_events.dtypes

In [ ]:
fact_events.info()

In [ ]:
fact_events.describe()

In [ ]:
fact_events.isna().sum()

<h2> Data preprocessing and Analysing</h2>

In [ ]:
# Find duplicate rows based on specific columns
duplicate_rows = fact_events.duplicated(subset=['store_id', 'campaign_id', 'product_code'], keep=False)

# Count how many duplicates exist
num_duplicates = duplicate_rows.sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Display the duplicated rows themselves
duplicated_values = fact_events[duplicate_rows].sort_values(by=['store_id', 'campaign_id', 'product_code'])
print(duplicated_values[["store_id", "campaign_id", "product_code"]])

In [ ]:
# Count duplicates before removing
num_duplicates_before = fact_events.duplicated(subset=['store_id', 'campaign_id', 'product_code']).sum()
print(f"Number of duplicate rows before removal: {num_duplicates_before}")

# Remove duplicates, keeping the first occurrence
fact_events = fact_events.drop_duplicates(
    subset=['store_id', 'campaign_id', 'product_code'], 
    keep='first'
)

# Count duplicates after removing
num_duplicates_after = fact_events.duplicated(subset=['store_id', 'campaign_id', 'product_code']).sum()
print(f"Number of duplicate rows after removal: {num_duplicates_after}")

# Number of duplicates removed
duplicates_removed = num_duplicates_before - num_duplicates_after
print(f"Number of duplicate rows removed: {duplicates_removed}")


In [ ]:
# Cities which have more than 5 Stores

stores_per_city = dim_stores.groupby("city")["store_id"].nunique()  # we are grouping store_id based on city

cities_more_than_5_stores = stores_per_city[stores_per_city>5]

num_of_cities = len(cities_more_than_5_stores)
city_names = cities_more_than_5_stores.index.tolist()

print(f"number of cities which has stores more than five is : {num_of_cities}")
print(f"\nCities with more than 5 stores: {city_names}")

In [ ]:
# Count missing values before imputation
num_missing_before = fact_events['quantity_sold(before_promo)'].isna().sum()
print(f"Missing values before imputation: {num_missing_before}")

# Calculate global median (ignoring NaNs)
median_value = fact_events['quantity_sold(before_promo)'].median()
print(f"Median used for imputation: {median_value}")

# Fill missing values with median
fact_events['quantity_sold(before_promo)'] = fact_events['quantity_sold(before_promo)'].fillna(median_value)

# Count missing values after imputation
num_missing_after = fact_events['quantity_sold(before_promo)'].isna().sum()
num_filled = num_missing_before - num_missing_after
print(f"Number of missing values filled: {num_filled}")

In [ ]:
print(fact_events.info())

In [ ]:
fact_events = fact_events.copy()

# Now you can safely calculate revenue columns
fact_events["revenue(before_promo)"] = (
    fact_events["base_price(before_promo)"] * fact_events["quantity_sold(before_promo)"]
)

fact_events["revenue(after_promo)"] = (
    fact_events["base_price(after_promo)"] * fact_events["quantity_sold(after_promo)"]
)

In [ ]:
# Merge to get store info (e.g., city)
fact_with_stores = fact_events.merge(
    dim_stores,  # dim_stores table
    on='store_id',  # key
    how='left'     # keep all fact rows
)


In [ ]:
# Merge to get product info (e.g., category, name)
fact_with_products = fact_with_stores.merge(
    dim_products,  # dim_products table
    on='product_code',  # key
    how='left'
)


In [ ]:
# Merge to get campaign info (e.g., campaign name, type)
total_fact_events = fact_with_products.merge(
    dim_campaigns,  # dim_campaigns table
    on='campaign_id',  # key
    how='left'
)

In [ ]:
fact_events[fact_events["event_id"] == "ba86f4"]

In [ ]:
total_fact_events.dtypes

In [ ]:
total_fact_events.info()

In [ ]:
total_fact_events.describe()

In [ ]:
total_fact_events.isna().sum()

In [ ]:
# Find duplicate rows based on specific columns
duplicate_rows = total_fact_events.duplicated(subset=['store_id', 'campaign_id', 'product_code'], keep=False)

# Count how many duplicates exist
num_duplicates = duplicate_rows.sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Display the duplicated rows themselves
duplicated_values = total_fact_events[duplicate_rows].sort_values(by=['store_id', 'campaign_id', 'product_code'])
print(duplicated_values[["store_id", "campaign_id", "product_code"]])

In [ ]:
category_min_price = total_fact_events.groupby("category")["base_price(before_promo)"].min() 

lowest_price_category = category_min_price.idxmin()
lowest_price = category_min_price.min()

print(f"Lowest base price: {lowest_price}")
print(f"\nProduct category with the lowest base price: {lowest_price_category}")

In [ ]:
BOGOF = total_fact_events["promo_type"] == "BOGOF"
Diwali = total_fact_events["campaign_name"] == "Diwali"

filterd_data = total_fact_events[BOGOF & Diwali]

total_Quantity_sold_after_promo = filterd_data["quantity_sold(after_promo)"].sum()

print(f"total quantity sold after promotion for BOGS during Diwali: {total_Quantity_sold_after_promo}")

In [ ]:
diwali_data = total_fact_events[total_fact_events["campaign_name"] == "Diwali"] # filtering the diwali dataa
store_sale = diwali_data.groupby("store_id")["quantity_sold(after_promo)"].sum().reset_index()
highest_store_quantity = store_sale.loc[store_sale["quantity_sold(after_promo)"].idxmax()]

print(f"Store with the highest quantity sold after Diwali promotion: {highest_store_quantity['store_id']}")
print(f"\nQuantity sold is: {highest_store_quantity['quantity_sold(after_promo)']}")

In [ ]:
campaign_sales = (
    total_fact_events[total_fact_events["campaign_name"].isin(["Sankranti", "Diwali"])]
    .groupby("campaign_name")[["quantity_sold(before_promo)", "quantity_sold(after_promo)"]]
    .sum()
    .reset_index()
)


In [ ]:
campaign_sales["increase_in_sales"] = (
    campaign_sales["quantity_sold(after_promo)"] - campaign_sales["quantity_sold(before_promo)"]
)

most_successful = campaign_sales.loc[campaign_sales["increase_in_sales"].idxmax()]

In [ ]:
print("📊 Total quantities sold before and after promotions:")
print(campaign_sales)

print("\n Campaign which has greater increase in sales:")
print(f"{most_successful['campaign_name']} — Increase: {most_successful['increase_in_sales']}")

In [ ]:
total_fact_events.head()

In [ ]:
totals = (
    total_fact_events
    .groupby(["campaign_name", "promo_type", "product_name"], as_index=False)
    .agg({
        "revenue(before_promo)": "sum",
        "revenue(after_promo)": "sum"
    })
)
totals = totals.rename(columns={
    "revenue(before_promo)": "revenue_before_total",
    "revenue(after_promo)": "revenue_after_total"
})

total_fact_events = total_fact_events.merge(
    totals,
    on=["campaign_name", "promo_type", "product_name"]
)

total_fact_events["IR%"] = (
    (total_fact_events["revenue_after_total"] - total_fact_events["revenue_before_total"])
    / total_fact_events["revenue_before_total"]
) * 100

total_fact_events.head()

In [ ]:
# Filter for Sankranti campaign safely
sankranti_data = total_fact_events[
    total_fact_events["campaign_name"].str.strip().str.lower() == "sankranti"
].copy()

# Find the product with the highest IR%
highest_ir_product = sankranti_data.loc[sankranti_data["IR%"].idxmax()]

print(f"Product with highest IR% during Sankranti: {highest_ir_product['product_name']}")
print(f"IR% for this product: {highest_ir_product['IR%']:.2f}%")

In [ ]:
totals_units = (
    total_fact_events
    .groupby(["campaign_name", "promo_type", "product_name"], as_index=False)
    .agg({
        "quantity_sold(before_promo)": "sum",
        "quantity_sold(after_promo)": "sum"
    })
)
totals_units = totals_units.rename(columns={
    "quantity_sold(before_promo)": "qty_before_total",
    "quantity_sold(after_promo)": "qty_after_total"
})

total_fact_events = total_fact_events.merge(
    totals_units,
    on=["campaign_name", "promo_type", "product_name"]
)

total_fact_events["ISU%"] = (
    (total_fact_events["qty_after_total"] - total_fact_events["qty_before_total"])
    / total_fact_events["qty_before_total"]
) * 100

total_fact_events.head()

In [ ]:
diwali_visakhapatnam = total_fact_events[
    (total_fact_events['campaign_name'] == 'Diwali') &
    (total_fact_events['city'] == 'Visakhapatnam')
]

store_totals = diwali_visakhapatnam.groupby('store_id', as_index=False).agg({
    'quantity_sold(before_promo)': 'sum',
    'quantity_sold(after_promo)': 'sum'
})

store_totals['ISU%'] = (
    (store_totals['quantity_sold(after_promo)'] - store_totals['quantity_sold(before_promo)'])
    / store_totals['quantity_sold(before_promo)']
) * 100

lowest_store = store_totals.loc[store_totals['ISU%'].idxmin()]

store_name = lowest_store['store_id']
lowest_ISU = lowest_store['ISU%']

print(f"Store with lowest ISU%: {store_name}")
print(f"ISU% for that store: {lowest_ISU:.2f}%")

In [ ]:
sankranti_data = total_fact_events[
    total_fact_events['campaign_name'] == 'Sankranti'
]

promo_aggregates = sankranti_data.groupby('promo_type', as_index=False, group_keys=False).apply(
    lambda x: pd.Series({
        'IR%': ( (x['IR%'] * x['revenue(before_promo)']).sum() ) / x['revenue(before_promo)'].sum(),
        'ISU%': ( (x['ISU%'] * x['quantity_sold(before_promo)']).sum() ) / x['quantity_sold(before_promo)'].sum()
    })
)
negative_promos = promo_aggregates[
    (promo_aggregates['IR%'] < 0) &
    (promo_aggregates['ISU%'] < 0)
]

print(negative_promos)

In [ ]:
store_count = (
    total_fact_events.groupby("city")["store_id"]
    .nunique()
    .reset_index(name="num_stores")
    .sort_values(by="num_stores", ascending=False)
)

plt.figure(figsize=(8, 4))
plt.bar(store_count["city"], store_count["num_stores"])
plt.title("Number of Stores by City", fontsize=14)
plt.xlabel("City", fontsize=12)
plt.ylabel("Number of Stores", fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis="y",linestyle=" ", alpha=0.7)
plt.show()

In [ ]:
top_city = store_count.iloc[0]
print(f"The city with the most stores is {top_city['city']} with {top_city['num_stores']} stores.")

In [ ]:
comparison = store_count[
    store_count["city"].isin(["Bengaluru", "Hyderabad", "Chennai"])
]
print(comparison)

In [ ]:
category_sales = (
    sankranti_data.groupby("category", as_index=False)["quantity_sold(after_promo)"]
    .sum()
    .rename(columns={"quantity_sold(after_promo)": "total_qty_after_promo"})
)

In [ ]:
total_sales = category_sales["total_qty_after_promo"].sum()
category_sales["percentage_contribution"] = (
    category_sales["total_qty_after_promo"] / total_sales * 100
)

max_index = category_sales["total_qty_after_promo"].idxmax()


explode = [0.1 if i == max_index else 0 for i in range(len(category_sales))]
plt.figure(figsize=(6, 6))
plt.pie(
    category_sales["total_qty_after_promo"],
    labels=category_sales["category"],
    autopct="%1.1f%%",
    startangle=200,
    counterclock= True,
    explode=explode,
    shadow=True,
)
plt.title("Sankranti Campaign - Quantity Sold After Promotion by Product Category", fontsize=14)
plt.show()

In [ ]:
corr_data = total_fact_events[["base_price(after_promo)", "quantity_sold(after_promo)"]]
correlation_matrix = corr_data.corr()
print(correlation_matrix)

plt.figure(figsize=(6, 4))
sns.heatmap(
    correlation_matrix,
    annot=True,
    cmap="coolwarm",
    fmt=".2f",
    linewidths=0.5
)
plt.title("Correlation between Base Price (After Promo) and Quantity Sold (After Promo)")
plt.show()

In [ ]:
data = total_fact_events[["category", "quantity_sold(before_promo)"]]


categories = data["category"].unique()

plt.figure(figsize=(15, 10))
for i, category in enumerate(categories, 1):
    plt.subplot(3, 3, i)
    subset = data[data["category"] == category]
    plt.hist(subset["quantity_sold(before_promo)"], bins=20, color='skyblue', edgecolor='black')
    plt.title(category)
    plt.xlabel("Quantity Sold (Before Promo)")
    plt.ylabel("Frequency")

plt.tight_layout()
plt.suptitle("Distribution of Quantity Sold Before Promotion by Product Category", fontsize=16, y=1.02)
plt.show()


In [ ]:
city_sales = (
    total_fact_events.groupby("city", as_index=False)
    .agg({
        "quantity_sold(before_promo)": "sum",
        "quantity_sold(after_promo)": "sum"
    })
    .rename(columns={
        "quantity_sold(before_promo)": "qty_before_total",
        "quantity_sold(after_promo)": "qty_after_total"
    })
)

# Step 2: Compute ISU% per city
city_sales["ISU%"] = (
    (city_sales["qty_after_total"] - city_sales["qty_before_total"])
    / city_sales["qty_before_total"]
) * 100

# Step 3: Sort by ISU% for visualization
city_sales = city_sales.sort_values("ISU%", ascending=False)

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(
    city_sales["city"],
    city_sales["ISU%"],
    marker="o",
    linestyle="-",
    color="mediumseagreen",
    linewidth=2,
    markersize=8
)

plt.title("Incremental Sold Units Percentage (ISU%) Across Cities", fontsize=14)
plt.xlabel("City")
plt.ylabel("ISU% (Incremental Sold Units %)")
plt.xticks(rotation=45)
plt.grid(True, linestyle=" ", alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
hyd_data = total_fact_events[total_fact_events["city"] == "Hyderabad"]
promo_summary = (
    hyd_data.groupby("promo_type", as_index=False)
    .agg({
        "IR%": "mean",
        "ISU%": "mean"
    })
    .sort_values("ISU%", ascending=False)
)
promo_summary

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(
    promo_summary["ISU%"],
    promo_summary["IR%"],
    s=150,
    color="royalblue",
    alpha=0.7,
    edgecolor="black"
)

for i, row in promo_summary.iterrows():
    plt.text(
        row["ISU%"] + 0.5,
        row["IR%"] + 0.5,
        row["promo_type"],
        fontsize=9,
        fontweight="bold"
    )

plt.title("Relationship between Incremental Revenue and Sold Units (Hyderabad)", fontsize=14)
plt.xlabel("Incremental Sold Units % (ISU%)")
plt.ylabel("Incremental Revenue % (IR%)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()


In [ ]:
bengaluru_data = total_fact_events[total_fact_events["city"] == "Bengaluru"]
category_revenue = bengaluru_data.groupby("category", as_index=False).agg({
    "revenue(before_promo)": "sum",
    "revenue(after_promo)": "sum"
})

category_revenue

In [ ]:
x = np.arange(len(category_revenue["category"]))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(x - width/2, category_revenue["revenue(before_promo)"], width, label="Revenue Before Promo", color="skyblue")
plt.bar(x + width/2, category_revenue["revenue(after_promo)"], width, label="Revenue After Promo", color="mediumseagreen")

plt.xlabel("Product Category", fontsize=12)
plt.ylabel("Revenue", fontsize=12)
plt.title("Revenue Before and After Promotions by Product Category (Bengaluru)", fontsize=14)
plt.xticks(x, category_revenue["category"], rotation=45)
plt.legend()
plt.grid(axis="y", linestyle=" ", alpha=0.5)
plt.tight_layout()
plt.show()